In [1]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup as bs
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from urllib.parse import urljoin
import json
import requests
import datetime

In [2]:
#driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# Ligas

**Esta función regresa un diccionario con el nombre de los equipos, su id y su slug.
Recibe el país, el slug y id del torneo del cuál se van a registrar los equipos**

In [ ]:
def liga_dict(country,slug,id):
    url = "https://www.sofascore.com/tournament/football/"+country+"/"+slug+"/"+id
    page = requests.get(url)
    soup1 = bs(page.text)
    script_tag = soup1.find_all('script',{'type':'application/json'})
    json2= script_tag[0].string
    data = json.loads(json2)
    js1 = data.get('props', {}).get('pageProps',{}).get('initialProps',{}).get('standings')
    js1=js1[0]
    eqs=js1.get('rows')
    equipos = {'teamName':[i.get('team',{}).get('name') for i in eqs], 'id':[i.get('team',{}).get('id') for i in eqs],'teamslug':[i.get('team',{}).get('slug') for i in eqs],"teamorigin":[country for i in eqs]}
    return equipos

**Esta función regresa un diccionario con los datos de los jugadores del equipo el cuál se recibieron los datos.**

In [39]:
def equipo(team,id,slug,pais):
    url = "https://www.sofascore.com/team/football/"+slug+"/"+str(id)
    page = requests.get(url)
    soup1 = bs(page.text)
    script_tag = soup1.find_all('script', {'type': 'application/ld+json'})
    x=script_tag[1]
    json_data = x.string
    data = json.loads(json_data)
    athletes = data.get('athlete', [])
    pretty_json_string = json.dumps(athletes)
    print(pretty_json_string)
    df = {'playerName':[i.get('name') for i in athletes], 'playercountry':[i.get('nationality', {}).get('name') for i in athletes],'team':[team for i in athletes],'teamid':[id for i in athletes],'teamorigin':[pais for i in athletes]}
    return df

**La función merge es para juntar dos diccionarios.**

In [5]:
def merge(dict1, dict2):
    # Merge dictionaries (concatenate lists)
    merged = {}
    for key in dict1:
        # Convert non-list values to single-item lists
        val1 = dict1[key] if isinstance(dict1[key], list) else [dict1[key]]
        val2 = dict2[key] if isinstance(dict2[key], list) else [dict2[key]]
        merged[key] = val1 + val2
    
    return merged

**La función liga_completa recibe un DataFrame, puede ser el de una liga, o muchas, pero es importante que tenga los datos importantes del equipo del que se quieren extraer datos, regres un diccionario.**

In [6]:
def liga_completa(dict):
    long = len(dict)
    df = {"playerName": [],"playercountry": [],"team": [],"teamid": [],"teamorigin":[]}
    for i in range(long):
        e1 = equipo(dict.iloc[i,0],dict.iloc[i,1],dict.iloc[i,2],dict.iloc[i,3])
        df = merge(df,e1)
    return df

**Esta función es para extraer los datos de los equipos de las top 30 ligas en el mundo, pero bien puede ser de muchos otros países.**

In [7]:
def todo(ligas):
    df = {"teamName": [],"id": [],"teamslug": [],"teamorigin":[]}
    y = len(ligas)
    for i in range(y):
        x = liga_dict(ligas.iloc[i,0],ligas.iloc[i,3],str(ligas.iloc[i,2]))
        df = merge(df,x)
    return df

# Paises

In [8]:
slugs = [
    "mexico", "england", "spain", "germany", "italy", "france", "afghanistan",
    "albania", "algeria", "american-samoa", "andorra", "angola", "anguilla", 
    "antigua-and-barbuda","argentina", "armenia", "aruba", "australia",
    "austria", "azerbaijan", "bahamas", "bahrain", "bangladesh", "barbados",
    "belarus", "belgium", "belize", "benin", "bermuda", "bhutan", "bolivia",
    "bosnia-herzegovina", "botswana", "brazil", "british-virgin-islands",
    "brunei", "bulgaria", "burundi", "cambodia", "cameroon", "canada",
    "cape-verde", "cayman-islands", "chad", "chile", "china", "chinese-taipei",
    "colombia", "comoros", "congo-republic", "cook-islands", "costa-rica",
    "croatia", "curacao", "cyprus", "czech-republic", "denmark", "dominica",
    "dominican-republic", "dr-congo", "ecuador", "egypt", "el-salvador",
    "estonia", "ethiopia", "faroe-islands", "fiji", "finland",
    "french-guiana", "gabon", "gambia", "georgia", "ghana", "gibraltar",
    "greece", "greenland", "grenada", "guadeloupe", "guam", "guatemala",
    "guinea", "honduras", "hong-kong", "hungary", "iceland", "india",
    "indonesia", "iran", "iraq", "ireland", "israel", "ivory-coast", "jamaica",
    "japan", "jordan", "kazakhstan", "kenya", "kosovo", "kuwait", "kyrgyzstan",
    "laos", "latvia", "lebanon", "lesotho", "liberia", "libya", "liechtenstein",
    "lithuania", "luxembourg", "macao", "madagascar", "malaysia", "malta",
    "martinique", "mauritania", "mauritius", "mayotte", "moldova", "mongolia",
    "montenegro", "morocco", "mozambique", "myanmar", "namibia", "nepal",
    "netherlands", "new-caledonia", "new-zealand", "nicaragua", "nigeria",
    "north-macedonia", "northern-ireland", "northern-mariana-islands",
    "norway", "pakistan", "palestine", "panama", "papua-new-guinea",
    "paraguay", "peru", "philippines", "poland", "portugal",
    "romania", "russia", "rwanda", "saint-kitts-and-nevis", "saint-martin",
    "saint-vincent-and-the-grenadines", "samoa", "san-marino",
    "sao-tome-and-principe", "scotland", "senegal", "serbia",
    "seychelles", "sierra-leone", "singapore", "sint-maarten", "slovakia",
    "slovenia", "solomon-islands", "somalia", "south-africa", "south-korea",
    "south-sudan", "suriname", "sweden", "switzerland", "tahiti",
    "tajikistan", "tanzania", "thailand", "tonga",
    "trinidad-and-tobago", "tunisia", "turkey", "turkmenistan",
    "turks-and-caicos-islands", "uganda", "ukraine", "united-arab-emirates"
    , "us-virgin-islands", "usa", "uzbekistan", "vanuatu", "venezuela",
    "vietnam", "wales", "zambia", "zimbabwe",
    "qatar","swaziland","oman","saudi-arabia","syria","uruguay"
]

**Esta función es para extraer los id's de las ligas de todos los países, probablemente no se vuelva a usar.**

In [9]:
def ligas_por_pais(pa,torn,id,slugs_t,pais):
    driver.get("https://www.sofascore.com/football/"+pais)
    cont1 = driver.page_source
    soup1 = bs(cont1)
    todo = soup1.find_all('a')
    for i in todo:
        soup = i.get("href")
        divi = soup.split("/")
        if (divi[1] == 'tournament') and (divi[2] == 'football') and (divi[3] == pais):
            h3=i.find('h3')
            if h3:
                nombre = h3.get_text(strip=True)
            else:
                continue
            pa+=[pais]
            torn+=[nombre]
            id+=[divi[5]]
            slugs_t+=[divi[4]]
    return pa,torn,id,slugs_t

**Esta función es para extraer los datos de las primeras divisiones de todos los países.**

In [10]:
pa=[]
torn=[]
id=[]
slugs_t=[]

def prims_div():
    for i in slugs:
        filt = dicc[dicc["slug_country"]==i]
        xf = filt.iloc[0]
        pa.append(xf.slug_country)
        torn.append(xf.tournament)
        id.append(xf.id)
        slugs_t.append(xf.slug_torn)
    prdiv={'slug_country':pa,'tournament':torn,'id':id,'slug_torn':slugs_t}
    return prdiv

**Llama a las funciones y extrae solo las primeras divisiones
Actualización: Una vez que ya estén definidas, puede que ya no las use**

dicc = pd.read_csv("ligas.csv")
prdiv = prims_div()
prdiv = pd.DataFrame(prdiv)
prdiv = prdiv.drop([18,30,106,169,182,144,44],axis =0)

In [11]:
prdiv = pd.read_csv("primeras_divisiones.csv")

**Aquí es donde se escogen solo las top 30 primeras divisiones, y se guardan en la variable prdiv_f, y puede que ya no sea nececario**

eqs = pd.read_csv("equipos_30.csv")
eqs=eqs.drop(["Unnamed: 0"], axis=1)

**Esta parte llama a la función todo, la cuál trae los equipos y sus datos de las ligas que sean el input, pued que también ya no se ocupe**

todos_equipos = todo(prdiv)
todos_equipos = pd.DataFrame(todos_equipos)

**En esta parte llamo a la función liga completa, la cuál no necesariamente recibe una liga, recibe un dataframe
con los equipos, y en este caso son todos, igual puede que ya no se use.**

jugadores = liga_completa(todos_equipos)
jugadores = jugadores.drop(["teamorigin","team","Unnamed: 0"], axis=1)

**Esta parte puede que ya no la use ya que es para renombrar las columnas de id por simple comodidad**

ligas = pd.read_csv("ligas.csv")
prdiv = f.rename(columns={"id":"leagueid"})
prdiv = f.rename(columns={"id":"teamid"})
ligas=ligas.rename(columns={"id":"leagueid"})
prdiv = pd.merge(prdiv,ligas,left_on="teamorigin",right_on="slug_country",how="inner")
prdiv=prdiv[prdiv["slug_torn"]=="liga-profesional-de-futbol"]
#prdiv[prdiv["slug_torn"]=="mls"].to_csv("usa.csv",index=False)
prdiv=prdiv[["slug_country","tournament","leagueid","slug_torn","teamName","teamid","teamslug","teamorigin"]]
prdiv.to_csv("arg.csv",index=False)

jugadores = g.drop(["teamorigin","team"], axis=1)

In [12]:
jugadores = pd.read_csv("jugadores.csv")
todos_equipos = pd.read_csv("equipos_primeras_divisiones.csv")

In [13]:
jugadores["teamid"] = jugadores["teamid"].astype(str)   # convertir a string
todos_equipos["teamid"] = todos_equipos["teamid"].astype(str)

**Este merge es para que se junten los equipos, y su país de origen**

In [14]:
#merge_p_equipos = pd.merge(prdiv,col, left_on="slug_country",right_on="teamorigin",how="inner")

**Este otro es para juntar las tablas de jugadores con equipos, no la convierto en un csv, es solo para el JSON.**

In [15]:
morg_prueba = pd.merge(jugadores,todos_equipos,left_on="teamid",right_on="teamid",how="right")

In [16]:
#morg["playercountry"] = morg["playercountry"].fillna("na")

In [17]:
import datetime
import json

def create_league_reports(df):
    reports = {}

    # Agrupar por liga: id, torneo y país
    for (leagueid, tournament, slug_country), league_df in df.groupby(["leagueid", "tournament", "slug_country"]):
        # Contar jugadores por nacionalidad
        country_counts = league_df["playercountry"].value_counts().to_dict()
        total_players = sum(country_counts.values())

        # Calcular cantidad de equipos (ids únicos)
        team_count = league_df["teamid"].nunique()

        # Lista de nacionalidades
        nationalities = [
            {
                "country": {"name": country},
                "playerCount": int(count),
                "percentage": round(int(count) / total_players * 100, 1) if total_players > 0 else 0
            }
            for country, count in country_counts.items()
        ]

        # Reporte de liga
        report = {
            "tournament": {
                "name": tournament,
                "slug": tournament.lower().replace(" ", "-"),
                "category": {
                    "name": "Football",
                    "slug": "football",
                    "sport": {"name": "Football", "slug": "football", "id": 1}
                },
                "country": {
                    "name": slug_country.capitalize(),
                    "slug": slug_country
                },
                "id": int(leagueid)
            },
            "type": "nationality_distribution",
            "nationalities": nationalities,
            "totalPlayers": int(total_players),
            "teamCount": int(team_count),
            "sufficientPlayers": bool(total_players >= team_count * 23),
            "updatedAt": datetime.datetime.now().isoformat()
        }

        # Guardar por país en vez de por nombre de torneo
        if slug_country not in reports:
            reports[slug_country] = []
        reports[slug_country].append(report)

    return reports


# Generar reporte
reports = create_league_reports(morg_prueba)

# Guardar en JSON
with open("league_reports.json", "w", encoding="utf-8") as f:
    json.dump(reports, f, indent=2, ensure_ascii=False)